### Welcome to the ProtoSyn.jl examples

# 6 - Steepest Descent

ProtoSyn makes available the Steepest Descent Driver, whose simulation algorithm calculates the forces being felt on each atom of a molecular structure (via an Energy Function instance) and updates the atoms position in accordance, as to relax the structure. In this example, we will relax the 2A3D peptide using the Steepest Descent Driver.

In [1]:
using ProtoSyn

 | Loading TorchANI
 | Loading ONNX models
 | Loading SeqDes
[ Loading: ProtoSyn loaded successfully!

.      ____            _       ____              
      |  _ \ _ __ ___ | |_ ___/ ___| _   _ _ __  
      | |_) | '__/ _ \| __/ _ \___ \| | | | '_ \ 
      |  __/| | | (_) | || (_) |__) | |_| | | | |
      |_|   |_|  \___/ \__\___/____/ \__, |_| |_|
                                       |_/       
    
      ---------------------------------------------

 Version      : 1.10
 License      : GNU-GPL-3
 Developed by : José Pereira (jose.manuel.pereira@ua.pt)
                Sérgio Santos


ProtoSyn will now display warnings and error messages. Debug: ⨯ | Info: ⨯ | Warnings: ✓ | Errors: ✓


1. Load the 2A3D structure.

The assignment of the default atom names and charges is not actually necessary for this example, but it's a good practice for when electrostatics are being calculated.

In [ ]:
pose = ProtoSyn.Peptides.load("data/2a3d.pdb")
ProtoSyn.Peptides.assign_default_atom_names!(pose)
ProtoSyn.Peptides.Calculators.Electrostatics.assign_default_charges!(pose);

2. Define the energy function

For this example, we'll minimize the TorchANI ML model energy (while maintaining a bond distance restraint). Other components, such as long-range electrostatics or all-atom clash restraints, can be added when necessary.

In [ ]:
torchani        = ProtoSyn.Calculators.TorchANI.get_default_torchani_model(α = 1.0)
bond_distance   = ProtoSyn.Calculators.Restraints.get_default_bond_distance_restraint(α = 1.0)
energy_function = ProtoSyn.Calculators.EnergyFunction([torchani, bond_distance])

3. Evaluate the initial state

In [ ]:
energy_function(pose)
ProtoSyn.write(pose, "output/example6.pdb")
display(pose.state.e)

4. Generate an alternative starting point

For the purpose of this example, the Backrub Mutator will quickly be employed to gently randomize the position of the atoms, in order to more visually veriy the energy minimization process. 

In [ ]:
scramble! = ProtoSyn.Mutators.BackrubMutator(ProtoSyn.rand_vector_in_sphere, 1.0, 0.2, nothing)
scramble!(pose)
energy_function(pose)
ProtoSyn.append(pose, "output/example6.pdb")
display(pose.state.e)

In [ ]:
callback = ProtoSyn.Common.default_energy_step_frame_callback(1, "output/example6.pdb")

In [ ]:
steepest_descent = ProtoSyn.Drivers.SteepestDescent(energy_function, callback, 500, 0.001, 0.1)

In [ ]:
steepest_descent(pose)

## Conclusion

In this quick example we explored the usage of the Steepest Descent Driver to relax a structure based on the forces being felt. This is usefull as a quick way to relax angles and bond distances in simulations, as well as fine tune the packaging of sidechains.